## 데이터 불러오기

<b>targetdata<br></b>
- Class: 0(사기가 아닌 정상적 신용카드 트랜잭션), 1(사기 트랜잭션)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


card_df = pd.read_csv('creditcard.csv')
card_df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## 데이터 전처리

### 데이터 일차 가공 후 모델 학습/예측/평가

#### 불필요한 피처 삭제

In [2]:
# 인자로 입력받은 데이터프레임을 복사하여 time 컬럼만 삭제하고 복사된 dataFrame 반환
def get_preprocessed_df(df = None):
    
    df_copy = df.copy()
    df_copy.drop('Time', axis = 1, inplace = True)
    
    return df_copy

In [3]:
from sklearn.model_selection import train_test_split

# 사전 데이터 가공 후 학습과 테스트 데이터 세트 반환하는 함수 생성
def get_train_test_dataset(df = None):
    # 데이터프레임의 사전 데이터 가공이 완료된 복사 데이터프레임 생성
    df_copy = get_preprocessed_df(df)
    
    # 데이터프레임의 피처, 레이블 나누기
    X_features = df_copy.iloc[:,:-1]
    y_target = df_copy.iloc[:,-1]
    
    # 학습/테스트 데이터 분할
    ## Stratified 방식으로 분할
    X_train, X_test, y_train, y_test = train_test_split(X_features, y_target,
                                                        test_size = 0.3,
                                                        random_state = 0,
                                                        stratify = y_target)
    
    #학습/테스트 데이터셋 변환
    return X_train, X_test, y_train, y_test

In [4]:
# 학습/테스트 데이터셋 불러오기
X_train, X_test, y_train, y_test = get_train_test_dataset(card_df)

# 학습/테스트 데이터 레이블 값 비율 확인
print('학습 데이터 비율')
print(y_train.value_counts() /y_train.shape[0] * 100)

print('테스트 데이터 비율')
print(y_test.value_counts() /y_test.shape[0] * 100)

학습 데이터 비율
0    99.827451
1     0.172549
Name: Class, dtype: float64
테스트 데이터 비율
0    99.826785
1     0.173215
Name: Class, dtype: float64


In [5]:
# 오차행렬, 정확도, 정밀도, 재현율, f1, AUC 평가 함수
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score

def get_clf_eval(y_test, pred, pred_proba):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))


#### 로지스틱 회귀 평가

In [6]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(max_iter = 1000)
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)
lr_pred_proba = lr_clf.predict_proba(X_test)[:,1]

get_clf_eval(y_test, lr_pred, lr_pred_proba)

오차 행렬
[[85281    14]
 [   56    92]]
정확도: 0.9992, 정밀도: 0.8679, 재현율: 0.6216,    F1: 0.7244, AUC:0.9704


테스트 데이터 세트로 측정시 재현율이 0.6149, roc-auc가 0.9702임

#### LGBM 평가

In [ ]:
def get_model_train_eval(model, ftr_train = None, ftr_test = None, tgt_train = None, tgt_test = None):
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    pred_proba = model.predict_proba(ftr_test)[:,1]
    
    get_clf_eval(tgt_test, pred, pred_proba)

In [ ]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators = 1000,
                          num_leaves = 64,
                          n_jobs = -1,
                          boost_from_average = False)

get_model_train_eval(lgbm_clf, X_train, X_test, y_train, y_test)

### 데이터 분포도 변환 후 모델 학습/예측/평가

#### 중요 피처의 분포도 살펴보기
- Amount 피처는 신용카드 사기금액으로 정상/사기 트랜잭션을 결정하는 매우 중요한 속성일 가능성이 높음

In [ ]:
# Amount 피처의 기술통계 확인

card_df['Amount'].describe()

In [ ]:
# Amount 피처의 분포도 확인해보기

import seaborn as sns
import matplotlib.pyplot as plt


plt.figure(figsize = (10,8))

# Amount 피처의 최솟값이 0, 최댓값을 고려해 범위 설정
plt.xticks(range(0, 300000, 1000), rotation = 60)

# 막대 개수 100개로 설정, kde 곡선그래프 보여주기
sns.histplot(card_df['Amount'], bins = 100, kde = True, color = 'red')

plt.show()

신용카드 사용금액이 1000불 이하인 데이터가 대부분임

##### 정규화하기

In [ ]:
from sklearn.preprocessing import StandardScaler

# amount 피처값 정규화 사용자함수 생성
def get_preprocessed_df(df = None):
     
    df_copy = df.copy()
    
    # 정규화
    scaler = StandardScaler()
    amount_n = scaler.fit_transform(df_copy['Amount'].values.reshape(-1,1))
    
    # 변환된 Amount를 데이터프레임 맨 앞 컬럼으로 입력
    df_copy.insert(0, 'Amount_Scaled', amount_n)
    
    # 기존의 Time, Amount 피처 삭제
    df_copy.drop(['Time', 'Amount'], axis = 1, inplace = True)
    
    return df_copy

In [ ]:
# amount 정규화
X_train, X_test, y_train, y_test = get_train_test_dataset(card_df)


# 로지스틱 회귀 및 LightGBM 수행
print('****로지스틱 회귀 예측 성능****')
lr_clf = LogisticRegression(max_iter = 1000)
get_model_train_eval(lr_clf, X_train, X_test, y_train, y_test)

print('\n****LightGBM 예측 성능(boost_from_average = True)****')
lgbm_clf = LGBMClassifier(n_estimators = 1000, num_leaves = 64, n_jobs = -1)
get_model_train_eval(lgbm_clf, X_train, X_test, y_train, y_test)

print('\n****LightGBM 예측 성능(boost_from_average = False)****')
lgbm_clf = LGBMClassifier(n_estimators = 1000, num_leaves = 64, n_jobs = -1,
                          boost_from_average = False)
get_model_train_eval(lgbm_clf, X_train, X_test, y_train, y_test)

<b><span style = 'background-color: #fff5b1'>boost_from_average</span></b>

boosting 계열 알고리즘은 최초로 만들어진 Tree에서 오류를 보정하면서 지속적으로 Tree를 update함<br>
이때 최초 Tree를 만들때 오류를 update하기 위해 어떤 score값을 만드는데 이때 타겟값의 특정 평균값을 사용할것인지 아닌지를 나타내는 파라미터로 알고 있음<br>

boost_from_average 파라미터의 기능상 imbalanced dataset의 이진 분류에서 그 하나의 파라미터로 인하여 성능을 그렇게 크게 저하시키는 것은 어렵게 때문에  파라미터의 특성 때문에 발생하는 현상이 아니라 일종의 버그로 간주해도 좋을 것 같다고 함

<b>기존 평가:</b>
```
오차 행렬
[[85290     5]
 [   36   112]]
정확도: 0.9995, 정밀도: 0.9573, 재현율: 0.7568,    F1: 0.8453, AUC:0.9790
```
로지스틱 회귀의 경우 정밀도와 재현율이 많이 저하됨
반면, LightGBM의 경우는 약간의 정밀도와 재현도가 저하되긴 했지만 큰 차이 없음


##### 로그변환하기

In [ ]:
import numpy as np

def get_processed_df(df = None):
    df_copy = df.copy()
    
    # 넘파이의 log1p()함수로 Amount 로그변환
    amount_n = np.log1p(df_copy['Amount'])
    
    # 변환된 Amount를 데이터프레임 맨 앞 컬럼으로 입력
    df_copy.insert(0, 'Amount_Scaled', amount_n)
    
    # 불필요한 컬럼 삭제
    df_copy.drop(['Time','Amount'], axis = 1, inplace = True)
    
    return df_copy

In [ ]:
# amount 로그변환
X_train, X_test, y_train, y_test = get_train_test_dataset(card_df)


# 로지스틱 회귀 및 LightGBM 수행
print('****로지스틱 회귀 예측 성능****')
lr_clf = LogisticRegression(max_iter = 1000)
get_model_train_eval(lr_clf, X_train, X_test, y_train, y_test)

print('\n****LightGBM 예측 성능(boost_from_average = True)****')
lgbm_clf = LGBMClassifier(n_estimators = 1000, num_leaves = 64, n_jobs = -1)
get_model_train_eval(lgbm_clf, X_train, X_test, y_train, y_test)

print('\n****LightGBM 예측 성능(boost_from_average = False)****')
lgbm_clf = LGBMClassifier(n_estimators = 1000, num_leaves = 64, n_jobs = -1,
                          boost_from_average = False)
get_model_train_eval(lgbm_clf, X_train, X_test, y_train, y_test)

<b>기존 평가:</b>
```
오차 행렬
[[85290     5]
 [   36   112]]
정확도: 0.9995, 정밀도: 0.9573, 재현율: 0.7568,    F1: 0.8453, AUC:0.9790
```
로지스틱 회귀의 경우 정밀도와 재현율이 많이 저하됨
반면, LightGBM의 경우는 약간의 정밀도와 재현도가 저하되긴 했지만 큰 차이 없음

<b><span style = 'background-color: #f5f0ff'>결론</span></b>

<span style = 'background-color: #f6f8fa'>레이블이 극도로 불균일한 데이터 세트에서 로지스틱 회귀는 데이터 변환 시 불완전한 성능 결과를 보여주고 있음</span>

### 이상치 데이터 제거 후 모델 학습/예측/평가

#### IQR을 활용하여 이상치 제거

In [ ]:
copy_df = get_preprocessed_df(card_df)

pd.DataFrame(round(copy_df.corr().iloc[:,-1],3))

In [ ]:
import seaborn as sns

plt.figure(figsize = (9,9))

# 히트맵그리기
# 음의 상관관계 높을수록 빨간색, 양의 상관관계 높을수록 파란색
sns.heatmap(card_df.corr(), cmap = 'RdBu')

v14, v17피처가 음이 상관관계가 높게 나타남

In [ ]:
# V14에 대해서 이상치 처리하기

import numpy as np

def get_outlier(df = None, column = None, weight = 1.5):
    
    # fraud에 해당하는 column 데이터만 추출
    fraud = df[df['Class'] == 1][column]
    
    quantile_25 = np.percentile(fraud.values, 25)
    quantile_75 = np.percentile(fraud.values, 75)
    
    # IQR을 구하고, IQR에 1.5를 곱해 최댓값과 최솟값 지점 구함
    iqr = quantile_75 - quantile_25
    iqr_weight = iqr * weight
    
    lowest_val = quantile_25 - iqr_weight
    highest_val = quantile_75 + iqr_weight
    
    # 최댓값보다 크거나, 최솟값보다 작은 값을 이상치 데이터로 설정
    outlier_index = fraud[(fraud > highest_val) | (fraud < lowest_val)].index
    
    return outlier_index

In [ ]:
outlier_index = get_outlier(df = card_df, column = 'V14', weight = 1.5)
print(outlier_index)

In [ ]:
# get_processed_df를 로그 변환 후 v14 피처의 이상치 데이터를 삭제하는 로직으로 변경

def get_preprocessed_df(df = None):
    df_copy = df.copy()
    
    # 로그변환 후 컬럼생성
    amount_n = np.log1p(df_copy['Amount'])
    df_copy.insert(0, 'Amount_Scaled', amount_n)
    
    # 불필요한 컬럼 삭제
    df_copy.drop(['Time','Amount'], axis = 1, inplace = True)
    
    # 이상치 데이터 삭제 로직 추가
    outlier_index = get_outlier(df = df_copy, column = 'V14', weight = 1.5)
    df_copy.drop(outlier_index, axis = 0, inplace = True)
    
    return df_copy

X_train, X_test, y_train, y_test = get_train_test_dataset(card_df)
print('****로지스틱 회귀 예측 성능****')
get_model_train_eval(lr_clf, X_train, X_test, y_train, y_test)

print('\n****LightGBM 예측 성능****')
get_model_train_eval(lgbm_clf, X_train, X_test, y_train, y_test)


<b>기존 성능:</b>
```
오차 행렬

[[85290     5]
 [   36   112]]
정확도: 0.9995, 정밀도: 0.9573, 재현율: 0.7568,    F1: 0.8453, AUC:0.9790
```

<b>로그변환 후 예측 성능:</b>
```
오차 행렬
[[85281    14]
 [   58    90]]
정확도: 0.9992, 정밀도: 0.8654, 재현율: 0.6081,    F1: 0.7143, AUC:0.9702


오차 행렬
[[85290     5]
 [   37   111]]
정확도: 0.9995, 정밀도: 0.9569, 재현율: 0.7500,    F1: 0.8409, AUC:0.9779
```

로지스틱 회귀, LightGBM 모두 정밀도와 재현율이 상승함



In [ ]:
from imblearn.over_sampling import SMOTE